# PIK3R1: Effects of Mutation

<b> Standard imports for playing with and plotting data frames. </b>

In [6]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

alt.renderers.enable('notebook')


RendererRegistry.enable('notebook')

<b> Import CPTAC </b>

In [7]:
import CPTAC

In [8]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

<b> List of proteins that interact with PIK3R1 (according to STRING) </b>

In [9]:
protList = ['IRS1', 'IRS2', 'ATK1', 'P55G', 'ERBB3', 'P85A', 'MRAS', 'PTEN', 'RPS6KB1']

<b> Proteome abundance of proteins that interact with PIK3R1 </b>

In [10]:
gene = 'PIK3CA'


pcutoff = 0.05/len(protList)
print(pcutoff)

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteinfindName = protein + "protein"
        proteindf.columns = [proteinfindName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteinfindName], wt[proteinfindName])
        print("Test for " + protein + ": ")
        print(ttest)

0.005555555555555556
Test for IRS1: 
Ttest_indResult(statistic=-0.5083089884962738, pvalue=0.6123789928887791)
Test for IRS2: 
Ttest_indResult(statistic=-1.968743995892205, pvalue=0.052165788180294895)
Test for ERBB3: 
Ttest_indResult(statistic=-1.387261936761837, pvalue=0.16850875255007064)
Test for MRAS: 
Ttest_indResult(statistic=-0.673913401424868, pvalue=0.5023089469279987)
Test for PTEN: 
Ttest_indResult(statistic=-1.3001667298575024, pvalue=0.1965933724559433)
Test for RPS6KB1: 
Ttest_indResult(statistic=-1.7815482303633114, pvalue=0.07792065877566294)


### Phosphoproteome abundance of interacting proteins

In [12]:
phosProtList = ['IRS1', 'ATK1', 'P55G', 'ERBB3', 'P85A', 'MRAS', 'PTEN', 'RPS6KB1']

In [14]:
gene = 'PIK3CA'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in phosProtList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)
print(pcutoff)
pcutoff = 1

for loc in genedf.columns:
    if not loc == gene:
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 

0.001851851851851852
Results for ERBB3-S686: 
Ttest_indResult(statistic=-1.6715990106369498, pvalue=0.09856021027975978)
Results for IRS1-S1005: 
Ttest_indResult(statistic=-0.9690977556578986, pvalue=0.3365203953002731)
Results for IRS1-S1100: 
Ttest_indResult(statistic=-2.144151151356816, pvalue=0.035056594434805)
Results for IRS1-S1101: 
Ttest_indResult(statistic=-2.098899731593709, pvalue=0.038979458432090074)
Results for IRS1-S1134: 
Ttest_indResult(statistic=-1.770797347381606, pvalue=0.08055195840145042)
Results for IRS1-S270: 
Ttest_indResult(statistic=-0.7860248714648148, pvalue=0.4337493578683552)
Results for IRS1-S312: 
Ttest_indResult(statistic=1.3429173455723695, pvalue=0.18561470344519224)
Results for IRS1-S323: 
Ttest_indResult(statistic=-0.5485686919133093, pvalue=0.5847228047850945)
Results for IRS1-S330: 
Ttest_indResult(statistic=-1.093938678842869, pvalue=0.27814694985592836)
Results for IRS1-S337: 
Ttest_indResult(statistic=0.4798182866151812, pvalue=0.6325746861136